In [1]:
import os, time
import fitsio
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
from astropy.table import vstack, Table, hstack

In [2]:
import seaborn as sns
sns.set(context='talk', style='ticks', font_scale=1.6)
%matplotlib inline

In [3]:
outdir = '/global/project/projectdirs/desi/users/ioannis/dr8-lslga'

In [4]:
def read_dr8_lslga(region='north', clobber=False):
    dr8dir = '/global/project/projectdirs/cosmo/work/legacysurvey/dr8'
    sweepdir = '/global/project/projectdirs/cosmo/work/legacysurvey/dr8/{}/sweep/8.0/'.format(region)
    
    outfile = os.path.join(outdir, 'dr8-lslga-{}.fits'.format(region))
    if os.path.isfile(outfile) and not clobber:
        print('Reading {}'.format(outfile))
        out = Table.read(outfile)
    else:
        lslgafile = os.path.join(os.getenv('LARGEGALAXIES_DIR'), 'LSLGA-v2.0.fits')
        print('Reading {}'.format(lslgafile))
        lslga = Table.read(lslgafile)

        out = []
        catfile = glob(os.path.join(sweepdir, 'sweep*.fits'))
        for ii, ff in enumerate(catfile):
            if ii % 50 == 0:
                print('{} / {}'.format(ii, len(catfile)))
            cc = Table(fitsio.read(ff, columns=['REF_CAT', 'REF_ID', 'RA', 'DEC', 'TYPE', 'FRACDEV',
                                                'SHAPEEXP_R', 'SHAPEEXP_E1', 'SHAPEEXP_E2', 
                                                'SHAPEDEV_R', 'SHAPEDEV_E1', 'SHAPEDEV_E2'], upper=True))
            ckeep = np.where(cc['REF_CAT'] == 'L2')[0]
            #keep = np.where([rcat.decode('utf-8').strip() == 'L2' for rcat in cc['REF_CAT']])[0]
            if len(ckeep) > 0:
                cc = cc[ckeep]
                cc = cc[np.argsort(cc['REF_ID'])] # sort!
                _, uu = np.unique(cc['REF_ID'], return_index=True)
                if len(uu) != len(cc):
                    print('Duplicate large galaxy in {}'.format(os.path.basename(ff)))
                    cc = cc[uu]
                lkeep = np.where(np.isin(lslga['LSLGA_ID'], cc['REF_ID']))[0]
                if len(lkeep) != len(cc):
                    print('Still a problem with duplicates!')
                lss = lslga[lkeep]
                assert(np.all(lss['LSLGA_ID'] == cc['REF_ID']))
                lss.rename_column('RA', 'RA_LSLGA')
                lss.rename_column('DEC', 'DEC_LSLGA')
                lss.rename_column('TYPE', 'MORPHTYPE')
                out.append(hstack((lss, cc)))
                #import pdb ; pdb.set_trace()
        out = vstack(out)
        out.write(outfile, overwrite=True)
    return out

In [5]:
%time north = read_dr8_lslga(region='north', clobber=True)

Reading /global/project/projectdirs/cosmo/staging/largegalaxies/v2.0/LSLGA-v2.0.fits
0 / 286
Duplicate large galaxy in sweep-220p030-230p035.fits
Duplicate large galaxy in sweep-160p045-170p050.fits
50 / 286
100 / 286
150 / 286
Duplicate large galaxy in sweep-110p045-120p050.fits
200 / 286
Duplicate large galaxy in sweep-220p040-230p045.fits
Duplicate large galaxy in sweep-240p050-250p055.fits
Duplicate large galaxy in sweep-200p035-210p040.fits
250 / 286
Duplicate large galaxy in sweep-260p040-270p045.fits
CPU times: user 2min 30s, sys: 31.2 s, total: 3min 2s
Wall time: 3min 38s


In [6]:
%time south = read_dr8_lslga(region='south', clobber=True)

Reading /global/project/projectdirs/cosmo/staging/largegalaxies/v2.0/LSLGA-v2.0.fits
0 / 437
Duplicate large galaxy in sweep-070m055-080m050.fits
Duplicate large galaxy in sweep-330m050-340m045.fits
50 / 437
Duplicate large galaxy in sweep-030m005-040p000.fits
100 / 437
Duplicate large galaxy in sweep-010p005-020p010.fits
Duplicate large galaxy in sweep-050m025-060m020.fits
150 / 437
Duplicate large galaxy in sweep-300m045-310m040.fits
Duplicate large galaxy in sweep-340m045-350m040.fits
Duplicate large galaxy in sweep-050m060-060m055.fits
200 / 437
Duplicate large galaxy in sweep-210p020-220p025.fits
Duplicate large galaxy in sweep-080m060-090m055.fits
Duplicate large galaxy in sweep-140p000-150p005.fits
Duplicate large galaxy in sweep-060m035-070m030.fits
Duplicate large galaxy in sweep-330m015-340m010.fits
250 / 437
Duplicate large galaxy in sweep-320m055-330m050.fits
Duplicate large galaxy in sweep-230p010-240p015.fits
300 / 437
Duplicate large galaxy in sweep-030p010-040p015.fits


In [7]:
stop

NameError: name 'stop' is not defined

In [ ]:
dra, ddec = (cat['RA'] - cat['RA_LSLGA']).data * 3600, (cat['DEC'] - cat['DEC_LSLGA']).data * 3600
print(np.median(dra), np.std(dra), np.median(ddec), np.std(ddec))
fig, ax = plt.subplots(figsize=(8, 6))
ax.scatter(dra, ddec, s=50)
ax.set_xlim(-40, 40)
ax.set_ylim(-40, 40)
ax.axhline(y=0, ls='-', alpha=0.8, color='k')
ax.axvline(x=0, ls='-', alpha=0.8, color='k')
ax.set_xlabel(r'$\Delta\,$(RA) (arcsec)')
ax.set_ylabel(r'$\Delta\,$(Dec) (arcsec)')
plt.subplots_adjust(bottom=0.15, left=0.15)
fig.savefig('lslga-radec.png')

In [ ]:
def get_e1e2(ba, phi):
    ab = 1. / ba
    e = (ab - 1) / (ab + 1)
    ee = -np.log(1 - e)
    angle = np.deg2rad(2. * (-phi))
    ee1 = ee * np.cos(angle)
    ee2 = ee * np.sin(angle)
    return ee1, ee2
        
def type2properties(cat, objtype):
    this = np.where([objtype == tt.strip() for tt in cat['TYPE']])[0]
    if objtype == 'EXP' or objtype == 'REX':
        reff, e1, e2 = cat['SHAPEEXP_R'][this], cat['SHAPEEXP_E1'][this], cat['SHAPEEXP_E2'][this]
    elif objtype == 'DEV':
        reff, e1, e2 = cat['SHAPEDEV_R'][this], cat['SHAPEDEV_E1'][this], cat['SHAPEDEV_E2'][this]
        
    lslga_rad = cat['D25'][this] / 2 * 60 / 2 # [arcmin]
    lslga_e1, lslga_e2 = get_e1e2(cat['BA'][this], cat['PA'][this])
        
    return reff, e1, e2, lslga_rad, lslga_e1, lslga_e2, cat[this]

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(18, 5))
#for objtype in ('COMP', 'DEV', 'EXP', 'PSF', 'REX'):
for objtype, marker in zip(('DEV', 'EXP', 'REX'), ('s', 'o', '*')):
    reff, e1, e2, lslga_rad, lslga_e1, lslga_e2, _ = type2properties(cat, objtype)
    ax[0].scatter(reff, lslga_rad, label=objtype, s=20, alpha=0.7, marker=marker)
    if objtype != 'REX':
        ax[1].scatter(e1, lslga_e1, s=20, alpha=0.7, marker=marker)
        ax[2].scatter(e2, lslga_e2, s=20, alpha=0.7, marker=marker)
ax[0].set_xlim(0, 40)
ax[0].set_ylim(0, 40)
ax[0].axhline(y=5, color='k')
ax[0].set_xlabel(r'r$_{eff}$ (Tractor, arcsec)')
ax[0].set_ylabel(r'0.5 * R(25) (LSLGA, arcsec)')
ax[1].set_xlabel(r'e$_1$ (Tractor)')
ax[1].set_ylabel(r'e$_1$ (LSLGA)')
ax[2].set_xlabel(r'e$_2$ (Tractor)')
ax[2].set_ylabel(r'e$_2$ (LSLGA)')
for xx in ax[1:]:
    xx.set_xlim(-1, 1)
    xx.set_ylim(-1, 1)
leg = ax[0].legend(loc='upper left', frameon=True, fontsize=16, markerscale=2)
for l in leg.get_lines():
    l.set_alpha(1)
for xx in ax:
    xx.plot(xx.get_xlim(), xx.get_ylim(), color='k', ls='--')
plt.subplots_adjust(wspace=0.35, bottom=0.2)
plt.savefig('check-lslga.png')

In [ ]:
reff, e1, e2, lslga_rad, lslga_e1, lslga_e2, devcat = type2properties(cat, 'DEV')
big = np.where(reff > 25)[0]
for cc in devcat[big][:20]:
    size = np.round(cc['D25'] * 1.5 * 60 / 0.262).astype(int)
    montagefile = 'montage/{}.jpg'.format(cc['GALAXY'].lower())
    jpgfile = []
    #print('http://legacysurvey.org/viewer-dev?ra={}&dec={}&zoom=14&layer=dr8b-decam&lslga'.format(cc['RA'], cc['DEC']))
    for ii, imtype in enumerate(('', '-model', '-resid')):
        jpgfile.append('jpg/{}{}.jpg'.format(cc['GALAXY'].lower(), imtype))
        url = '"http://legacysurvey.org/viewer-dev/jpeg-cutout?ra={}&dec={}&size={}&layer=dr8b-decam{}"'.format(
            cc['RA'], cc['DEC'], size, imtype)
        if not os.path.exists(jpgfile[ii]):
            cmd = 'wget --continue -O {} {}'.format(jpgfile[ii], url)
            print(cmd)
            os.system(cmd)
            time.sleep(1)
    cmd = 'montage -bordercolor white -borderwidth 1 -tile 3x1 -geometry +0+0 '
    cmd = cmd+' '.join(ff for ff in jpgfile)
    cmd = cmd+' {}'.format(montagefile)
    print(cmd)
    os.system(cmd)    